In [14]:
import copy 
# The eight movement directions possible for a chess queen
RAYS = [(1, 0), (1, -1), (0, -1), (-1, -1),
        (-1, 0), (-1, 1), (0, 1), (1, 1)]


In [15]:
class GameState:

    def __init__(self, xlen = 3, ylen=2):
        """The GameState class constructor performs required
        initializations when an instance is created. The class
        should:
        
        1) Keep track of which cells are open/closed
        2) Identify which player has initiative
        3) Record the current location of each player
        
        Parameters
        ----------
        self:
            instance methods automatically take "self" as an
            argument in python
        
        Returns
        -------
        None
        """
        self._xlen = xlen
        self._ylen = ylen
        self._board = ([[0] * self._ylen for x in range(self._xlen)])
        self._board[2][1] = 1
        self._player = 0 # id 0 for player 1, 1 for player 2
        self._locations = [None, None]
        
    def actions(self):
        """ Return a list of legal actions for the active player 
        
        You are free to choose any convention to represent actions,
        but one option is to represent actions by the (row, column)
        of the endpoint for the token. For example, if your token is
        in (0, 0), and your opponent is in (1, 0) then the legal
        actions could be encoded as (0, 1) and (0, 2).
        """
        return self.liberties(self._locations[self._player])
    
    def player(self):
        """ Return the id of the active player 
        
        Hint: return 0 for the first player, and 1 for the second player
        """
        return self._player
    
    def result(self, action):
        """ Return a new state that results from applying the given
        action in the current state
        
        Hint: Check out the deepcopy module--do NOT modify the
        objects internal state in place
        """
        assert action in self.actions()
        resultState = copy.deepcopy(self)
        resultState._board[action[0]][action[1]] = 1
        resultState._locations[self._player] = action
        resultState._player ^= 1 
        return resultState
    
    def terminal_test(self):
        """ return True if the current state is terminal,
        and False otherwise
        
        Hint: an Isolation state is terminal if _either_
        player has no remaining liberties (even if the
        player is not active in the current state)
        """
        global call_counter
        call_counter += 1
        return not any(self.liberties(self._locations[0])) or not any(self.liberties(self._locations[1]))
    
    def liberties(self, loc):
        """ Return a list of all open cells in the
        neighborhood of the specified location.  The list 
        should include all open spaces in a straight line
        along any row, column or diagonal from the current
        position. (Tokens CANNOT move through obstacles
        or blocked squares in queens Isolation.)
        
        Note: if loc is None, then return all empty cells
        on the board
        """
        if loc is None:
            return [(x,y) for y in range(self._ylen) for x in range(self._xlen) if not self._board[x][y]]
        moves = []
        for dx, dy in RAYS:  # check each movement direction
            _x, _y = loc
            while 0 <= _x + dx < self._xlen and 0 <= _y + dy < self._ylen:
                _x, _y = _x + dx, _y + dy
                if self._board[_x][_y]:  # stop at any blocked cell
                    break
                moves.append((_x, _y))
        return moves
    
    def _has_liberties(self, player_id):
        """ Check to see if the specified player has any liberties """
        return any(self.liberties(self._locations[player_id]))
    
    def utility(self, player_id):
        """ return +inf if the game is terminal and the
        specified player wins, return -inf if the game
        is terminal and the specified player loses, and
        return 0 if the game is not terminal
        """
        if not self.terminal_test(): return 0
        player_id_is_active = (player_id == self.player())
        active_has_liberties = self._has_liberties(self.player())
        active_player_wins = (active_has_liberties == player_id_is_active)
        return float("inf") if active_player_wins else float("-inf")

In [16]:
# TODO: Implement the my_moves() function
# TODO: Change the value returned when the depth cutoff is
#       reached to call and return the score from my_moves()

# Use the player_id when you call "my_moves()"
# DO NOT MODIFY THE PLAYER ID
player_id = 0

def my_moves(gameState):
    # TODO: Finish this function!
    # HINT: the global player_id variable is accessible inside
    #       this function scope
    return len(gameState.liberties(gameState._locations[player_id]))

In [31]:
# implement alpha beta algorithm root function 

def alpha_beta_search(gameState):
    """ Return the move along a branch of the game tree that
    has the best possible value.  A move is a pair of coordinates
    in (column, row) order corresponding to a legal move for
    the searching player.
    
    You can ignore the special case of calling this function
    from a terminal state.
    """
    alpha = float("-inf")
    beta = float("inf")
    best_score = float("-inf")
    best_move = None
    for a in gameState.actions():
        v = min_value(gameState.result(a), alpha, beta)
        alpha = max(alpha, v)
        if v > best_score:
            best_score = v
            best_move = a
    return best_move

In [32]:
# implementing MINIMAX algorithm helper functions with alpha beta parameters

def min_value(gameState, alpha, beta):
    """ Return the value for a win (+1) if the game is over,
    otherwise return the minimum value over all legal child
    nodes.
    """
    if gameState.terminal_test():
        return gameState.utility(0)
    
    v = float("inf")
    for a in gameState.actions():
        v = min(v, max_value(gameState.result(a), alpha, beta))
        if v <= alpha:
            return v
        beta = min(beta, v)
    return v

# TODO: modify the function signature to accept an alpha and beta parameter
def max_value(gameState, alpha, beta):
    """ Return the value for a loss (-1) if the game is over,
    otherwise return the maximum value over all legal child
    nodes.
    """
    if gameState.terminal_test():
        return gameState.utility(0)
    
    v = float("-inf")
    for a in gameState.actions():
        v = max(v, min_value(gameState.result(a), alpha, beta))
        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v




In [33]:
# Test the depth limit by checking the number of nodes visited
# -- recall that minimax visits every node in the search tree,
# so if we search depth one on an empty board then minimax should
# visit each of the five open spaces
call_counter = 0
expected_node_count = 55
rootNode = GameState()
alpha_beta_search(rootNode)

print("Expected node count: {}".format(expected_node_count))
print("Your node count: {}".format(call_counter))

if call_counter == expected_node_count:
    print("That's right! Looks like your alpha-beta pruning is working!")
else:
    print("Uh oh...looks like there may be a problem.")

Expected node count: 55
Your node count: 55
That's right! Looks like your alpha-beta pruning is working!
